In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.io import savemat
import os


In [ ]:
class OFDM_TX: 
    def __init__(self,ifft_size,cp_length,num_symbols,mod_order,num_active):
        self.ifft_size = ifft_size
        self.cp_length = cp_length
        self.num_symbols = num_symbols
        self.mod_order = mod_order
        self.num_active = num_active 

    def OFDM_Tx(self):
        Ld = self.ifft_size
        Ls = Ld + self.cp_length 
        dc_index = self.ifft_size // 2 
        half = self.num_active // 2
        active_subcarriers = list(range(dc_index - half, dc_index)) + \
                            list(range(dc_index + 1, dc_index + half + 1))
        return dc_index, active_subcarriers

    def generate_mqam_symbols(self):
        bits_per_symbol = int(np.log2(self.mod_order))
        total_bits = self.num_active * self.num_symbols * bits_per_symbol
        bits = np.random.randint(0,2,total_bits) 
        symbol_bin = bits.reshape(-1,bits_per_symbol) 
        bits_per_axis = bits_per_symbol // 2 
        M_axis = int(np.sqrt(self.mod_order)) 
        bits_I = symbol_bin[:, :bits_per_axis] 
        bits_Q = symbol_bin[:, bits_per_axis:] 
        def bin2dec(arr): 
            return arr.dot(1 << np.arange(arr.shape[-1]-1,-1,-1))
        I_indices = bin2dec(bits_I)
        Q_indices = bin2dec(bits_Q) 
        I_vals = 2*I_indices - (M_axis - 1)
        Q_vals = 2*Q_indices - (M_axis - 1)
        qam_data = I_vals + 1j*Q_vals 
        qam_data /= np.sqrt(np.mean(np.abs(qam_data)**2))

        return qam_data, bits 

    def generate_ofdm_signal(self, qam_data, active_subcarriers, dc_index):
        self.num_active = len(active_subcarriers)
        symbol_matrix = np.zeros((num_symbols, ifft_size), dtype=complex)
        ofdm_signal = []

        for i in range(num_symbols):
            idx_start = i * num_active
            idx_end = (i + 1) * num_active
            this_symbol = qam_data[idx_start:idx_end]
            freq_data = np.zeros(ifft_size, dtype=complex)
            freq_data[active_subcarriers] = this_symbol
            freq_data[dc_index] = 0  # Null DC
            time_data = np.fft.ifft(freq_data) * np.sqrt(ifft_size)
            symbol_matrix[i, :] = freq_data
            cp = time_data[-cp_length:]
            ofdm_symbol = np.concatenate((cp, time_data))
            ofdm_signal.append(ofdm_symbol)

        ofdm_signal = np.concatenate(ofdm_signal)
        ofdm_signal /= np.sqrt(np.mean(np.abs(ofdm_signal) ** 2))

        return ofdm_signal, symbol_matrix

    def add_noise_and_pad(ofdm_signal, noise_snr_db, left_pad, right_pad, save_path='OFDM_Rx_Signal.mat'):
        signal_power = np.mean(np.abs(ofdm_signal)**2)
        noise_power = signal_power / (10**(noise_snr_db / 10))
        noise_std = np.sqrt(noise_power / 2)
        inband_noise = noise_std * (np.random.randn(len(ofdm_signal)) + 1j * np.random.randn(len(ofdm_signal)))
        ofdm_signal_noisy = ofdm_signal + inband_noise
        noise_before = noise_std * (np.random.randn(left_pad) + 1j * np.random.randn(left_pad))
        noise_after  = noise_std * (np.random.randn(right_pad) + 1j * np.random.randn(right_pad))
        final_signal = np.concatenate([noise_before, ofdm_signal_noisy, noise_after])
        savemat(save_path, {'final_signal': final_signal})

        return final_signal






ifft_size = 720 # FFT size = number of subcarriers 
cp_length = 80
num_symbols = 20
mod_order = 16 # modulation order (e.g., 16-QAM)
num_active = 109 # number of active subcarriers 

qam_data, bits = generate_mqam_symbols(mod_order, num_active, num_symbols)
print("QAM symbols shape:", qam_data.shape)  

ofdm_signal, symbol_matrix = generate_ofdm_signal(
    qam_data=qam_data,
    ifft_size=ifft_size,
    cp_length=cp_length,
    num_symbols=num_symbols,
    active_subcarriers=active_subcarriers,  
    dc_index=dc_index
)


